<a href="https://colab.research.google.com/github/JDK00/TP-SQLSPARK/blob/main/TPSparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#this one i used it just to upload the file into the claude
from google.colab import files
uploaded =files.upload()

Saving fifaworldcup.csv to fifaworldcup.csv


In [2]:

!pip install pyspark
from pyspark.sql import SparkSession

# Initialiser SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkSQLTP") \
    .getOrCreate()

In [3]:
# Charger les donnees du fichier CSV
df = spark.read.csv("fifaworldcup.csv", header=True, inferSchema=True)
df.show()


+----------+----------------+---------+----------+----------+----------+---------+--------+-------+
|      date|       home_team|away_team|home_score|away_score|tournament|     city| country|neutral|
+----------+----------------+---------+----------+----------+----------+---------+--------+-------+
|1872-11-30|        Scotland|  England|         0|         0|  Friendly|  Glasgow|Scotland|  false|
|1873-03-08|         England| Scotland|         4|         2|  Friendly|   London| England|  false|
|1874-03-07|        Scotland|  England|         2|         1|  Friendly|  Glasgow|Scotland|  false|
|1875-03-06|         England| Scotland|         2|         2|  Friendly|   London| England|  false|
|1876-03-04|        Scotland|  England|         3|         0|  Friendly|  Glasgow|Scotland|  false|
|1876-03-25|        Scotland|    Wales|         4|         0|  Friendly|  Glasgow|Scotland|  false|
|1877-03-03|         England| Scotland|         1|         3|  Friendly|   London| England|  false|


In [5]:
 #1.Combien de matchs sont présents dans le dataset ?
 # Enregistrement du DataFrame en tant que table SQL
df.createOrReplaceTempView("ma_table")

# Requête SQL pour compter
resultat = spark.sql("SELECT count(*) FROM ma_table")
resultat.show()

+--------+
|count(1)|
+--------+
|   44152|
+--------+



In [ ]:
#2. Quelle est la première et la dernière année couverte par les données ?
PreAnnée = spark.sql("SELECT min(year(date)) FROM ma_table")
PreAnnée.show()
DerAannée = spark.sql("SELECT max(year(date)) FROM ma_table")
DerAannée.show()


+---------------+
|min(year(date))|
+---------------+
|           1872|
+---------------+

+---------------+
|max(year(date))|
+---------------+
|           2022|
+---------------+



In [ ]:
#Lister les 10 tournois les plus fréquents
tournoisF = spark.sql("SELECT   tournament, COUNT(*) AS frequency FROM ma_table  GROUP BY tournament ORDER BY frequency DESC Limit 10")
tournoisF.show()

+--------------------+---------+
|          tournament|frequency|
+--------------------+---------+
|            Friendly|    17461|
|FIFA World Cup qu...|     7774|
|UEFA Euro qualifi...|     2593|
|African Cup of Na...|     1932|
|      FIFA World Cup|      948|
|        Copa América|      841|
|AFC Asian Cup qua...|      764|
|African Cup of Na...|      742|
|          CECAFA Cup|      620|
|CFU Caribbean Cup...|      606|
+--------------------+---------+



In [ ]:
#4. Combien de matchs ont été joués sur terrain neutre ?
nbrMatch = spark.sql("SELECT    COUNT(*)  FROM ma_table  where neutral=true ")
nbrMatch.show()

+--------+
|count(1)|
+--------+
|   10996|
+--------+



In [ ]:
#5. Lister les 10 pays ayant accueilli le plus de matchs.
nbrMatch = spark.sql("SELECT country,COUNT(*) AS frequency FROM ma_table group BY  country ORDER BY frequency DESC LIMIT 10")
nbrMatch.show()

+--------------------+---------+
|             country|frequency|
+--------------------+---------+
|       United States|     1259|
|              France|      830|
|            Malaysia|      752|
|             England|      722|
|              Sweden|      659|
|               Qatar|      618|
|             Germany|      608|
|              Brazil|      569|
|               Spain|      568|
|United Arab Emirates|      548|
+--------------------+---------+



In [ ]:
#6. Combien de matchs ont terminé sur un score nul ?
nbrMatch = spark.sql("SELECT COUNT(*) AS total_nuls FROM ma_table where home_score =away_score")
nbrMatch.show()

+----------+
|total_nuls|
+----------+
|     10213|
+----------+



In [ ]:
#7. Afficher les matchs où le score total (home + away) est supérieur à 6
nbrMatch = spark.sql("SELECT *  FROM ma_table where (cast(home_score as int) +cast(away_score as int))>6")
nbrMatch.show()

+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|      date|       home_team|       away_team|home_score|away_score|          tournament|     city| country|neutral|
+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|1878-03-02|        Scotland|         England|         7|         2|            Friendly|  Glasgow|Scotland|  false|
|1878-03-23|        Scotland|           Wales|         9|         0|            Friendly|  Glasgow|Scotland|  false|
|1879-04-05|         England|        Scotland|         5|         4|            Friendly|   London| England|  false|
|1880-03-13|        Scotland|         England|         5|         4|            Friendly|  Glasgow|Scotland|  false|
|1881-03-12|         England|        Scotland|         1|         6|            Friendly|   London| England|  false|
|1882-02-18|Northern Ireland|         England|         0|       

In [ ]:
#8. Nombre total de matchs joués par chaque équipe (domicile + extérieur).
nbrMatch = spark.sql(" select d.team, COUNT(m.away_team) AS away_count  from   ( SELECT distinct(home_team)as team  FROM ma_table  union all SELECT distinct(away_team )as team  FROM ma_table ) as d LEFT JOIN ma_table m ON d.team = m.away_team GROUP BY d.team ")
nbrMatch.show()

+-------------+----------+
|         team|away_count|
+-------------+----------+
|      Kabylia|        10|
|       Kernow|         6|
|         Chad|       112|
|     Provence|        18|
|       Russia|       810|
|     Paraguay|       946|
|     Anguilla|        90|
|        Yemen|       260|
|South Ossetia|        14|
|      Senegal|       662|
|       Madrid|         0|
|       Sweden|      1094|
|        Frøya|        36|
|     Kiribati|        12|
|     Ynys Môn|        64|
|       Guyana|       270|
|  Philippines|       274|
|       Jersey|        68|
|      Eritrea|        92|
|        Tibet|        18|
+-------------+----------+
only showing top 20 rows


In [ ]:
#9. Top 10 des équipes ayant marqué le plus de buts (toutes compétitions confondues).
nbrMatch = spark.sql(" SELECT team_name, SUM(goals_scored) AS total_goals FROM (SELECT home_team AS team_name, home_score AS goals_scored FROM ma_table where home_score !='NA'  UNION ALL SELECT away_team AS team_name, away_score AS goals_scored FROM ma_table where  away_score!='NA') AS combined_scores GROUP BY team_name ORDER BY total_goals DESC LIMIT 10 ")
nbrMatch.show()



+-----------+-----------+
|  team_name|total_goals|
+-----------+-----------+
|    England|     2282.0|
|     Brazil|     2228.0|
|    Germany|     2205.0|
|     Sweden|     2060.0|
|    Hungary|     1948.0|
|  Argentina|     1880.0|
|Netherlands|     1694.0|
|     Mexico|     1592.0|
|South Korea|     1576.0|
|     France|     1560.0|
+-----------+-----------+



In [ ]:
#10.Moyenne de buts par match par décennie.
nbrMatch = spark.sql("SELECT FLOOR(YEAR(CAST(date AS DATE)) / 10) * 10  AS decennie ,cast( avg( cast(home_score as int) + cast( away_score as int)) as int) FROM ma_table  where home_score !='NA'  or away_score!='NA'  group by decennie   ORDER BY decennie  ")
nbrMatch.show()


+--------+---------------------------------------------------------------------+
|decennie|CAST(avg((CAST(home_score AS INT) + CAST(away_score AS INT))) AS INT)|
+--------+---------------------------------------------------------------------+
|    1870|                                                                    4|
|    1880|                                                                    5|
|    1890|                                                                    5|
|    1900|                                                                    4|
|    1910|                                                                    4|
|    1920|                                                                    3|
|    1930|                                                                    4|
|    1940|                                                                    4|
|    1950|                                                                    3|
|    1960|                  

In [ ]:
#11.Nombre de matchs joués par tournoi et par année.
nbrMatch = spark.sql("SELECT  tournament,YEAR(date)as Annee ,count(*) as nbrmatch  FROM ma_table    group by tournament,Annee")
nbrMatch.show()

+--------------------+-----+--------+
|          tournament|Annee|nbrmatch|
+--------------------+-----+--------+
|     Copa Rio Branco| 1946|       1|
|Copa Bernardo O'H...| 1959|       2|
|   Copa Félix Bogado| 1983|       2|
|        Copa Artigas| 1985|       2|
|           Kirin Cup| 1993|       3|
|        Copa América| 2001|      26|
|   AFC Challenge Cup| 2014|      16|
|           Kirin Cup| 2016|       4|
|            Friendly| 2018|     426|
|AFF Championship ...| 2018|       2|
| Kirin Challenge Cup| 2021|       1|
|Copa Chevallier B...| 1926|       2|
|   International Cup| 1927|       4|
|Copa Bernardo O'H...| 1966|       2|
| Nordic Championship| 1972|       4|
|  Merdeka Tournament| 1974|      10|
|  Copa Paz del Chaco| 1980|       2|
|          King's Cup| 1980|       3|
|            Friendly| 1984|     242|
| Oceania Nations Cup| 2016|      15|
+--------------------+-----+--------+
only showing top 20 rows


In [ ]:
#12.Classement des équipes ayant remporté le plus de matchs à domicile.
nbrMatch = spark.sql("SELECT home_team AS equipe, COUNT(*) AS nombre_victoires_domicile FROM ma_table WHERE home_score > away_score GROUP BY home_team ORDER BY nombre_victoires_domicile DESC")
nbrMatch.show()

+-------------+-------------------------+
|       equipe|nombre_victoires_domicile|
+-------------+-------------------------+
|       Brazil|                      423|
|    Argentina|                      373|
|      Germany|                      327|
|      England|                      323|
|       Mexico|                      320|
|  South Korea|                      296|
|       Sweden|                      294|
|        Italy|                      291|
|       France|                      290|
|      Hungary|                      265|
|        Egypt|                      260|
|        Spain|                      256|
|  Netherlands|                      252|
|United States|                      243|
|      Belgium|                      229|
|      Denmark|                      229|
|     Scotland|                      224|
| Saudi Arabia|                      222|
|      Austria|                      220|
|        Chile|                      213|
+-------------+-------------------

In [ ]:
#13.Nombre de victoires, défaites et nuls pour chaque équipe.
nbrMatch = spark.sql("SELECT equipe, SUM(CASE WHEN score_equipe > score_adversaire THEN 1 ELSE 0 END) AS victoires, SUM(CASE WHEN score_equipe < score_adversaire THEN 1 ELSE 0 END) AS defaites, SUM(CASE WHEN score_equipe = score_adversaire THEN 1 ELSE 0 END) AS nuls FROM (SELECT home_team AS equipe, home_score AS score_equipe, away_score AS score_adversaire FROM ma_table UNION ALL SELECT away_team AS equipe, away_score AS score_equipe, home_score AS score_adversaire FROM ma_table) GROUP BY equipe ORDER BY victoires DESC")
nbrMatch.show()

+-----------+---------+--------+----+
|     equipe|victoires|defaites|nuls|
+-----------+---------+--------+----+
|     Brazil|      651|     160| 208|
|    England|      593|     201| 253|
|    Germany|      573|     205| 208|
|  Argentina|      547|     213| 254|
|     Sweden|      516|     309| 227|
|South Korea|      477|     192| 235|
|     Mexico|      467|     243| 216|
|    Hungary|      452|     301| 211|
|      Italy|      445|     157| 235|
|     France|      436|     249| 190|
|      Spain|      424|     134| 173|
|Netherlands|      419|     213| 188|
|    Uruguay|      399|     294| 226|
|   Scotland|      385|     253| 176|
|    Denmark|      378|     278| 176|
|     Russia|      365|     164| 187|
|     Poland|      365|     268| 218|
|    Belgium|      354|     277| 173|
|     Zambia|      346|     219| 201|
|    Austria|      340|     303| 177|
+-----------+---------+--------+----+
only showing top 20 rows


In [ ]:
#14.Score moyen des matchs joués sur terrain neutre vs non neutre.
nbrMatch = spark.sql("""
    SELECT
        cast(AVG(CASE WHEN neutral = true THEN cast (home_score as int) + cast(away_score as int) END) as int) AS neutre,
        cast(AVG(CASE WHEN neutral = false THEN cast(home_score as int) + cast(away_score as int) END) as int) AS non_neutre,
        cast(
            cast( AVG(CASE WHEN neutral = true THEN cast(home_score as int) + cast(away_score as int) END)  as int) -
           cast( AVG(CASE WHEN neutral = false THEN cast(home_score as int) + cast(away_score as int) END) as int )as int
        ) AS difference
    FROM ma_table  WHERE  home_score !='NA' AND away_score  !='NA'
""")
nbrMatch.show()

+------+----------+----------+
|neutre|non_neutre|difference|
+------+----------+----------+
|     3|         2|         1|
+------+----------+----------+



In [ ]:
#15.Top 5 des matchs avec l’écart de score le plus élevé.
nbrMatch = spark.sql("""
    SELECT  date,  home_team, away_team, home_score, away_score, ABS(CAST(home_score AS INT) - CAST(away_score AS INT)) AS ecart
    FROM ma_table
    WHERE home_score != 'NA' AND away_score != 'NA'
    ORDER BY ecart DESC
    LIMIT 5
""")
nbrMatch.show()

+----------+---------+--------------+----------+----------+-----+
|      date|home_team|     away_team|home_score|away_score|ecart|
+----------+---------+--------------+----------+----------+-----+
|2001-04-11|Australia|American Samoa|        31|         0|   31|
|1971-09-13|   Tahiti|  Cook Islands|        30|         0|   30|
|1979-08-30|     Fiji|      Kiribati|        24|         0|   24|
|2001-04-09|Australia|         Tonga|        22|         0|   22|
|1966-04-03|    Libya|          Oman|        21|         0|   21|
+----------+---------+--------------+----------+----------+-----+



In [ ]:
#16.Calculer le goal average (buts marqués − buts encaissés) par équipe.
nbrMatch = spark.sql("""
   SELECT
        equipe,
        SUM(buts_pour) AS total_marques,
        SUM(buts_contre) AS total_encaisses,
        (SUM(buts_pour) - SUM(buts_contre)) AS goal_average
    FROM (

        SELECT
            home_team AS equipe,
            CAST(home_score AS INT) AS buts_pour,
            CAST(away_score AS INT) AS buts_contre
        FROM ma_table
        WHERE home_score != 'NA' AND away_score != 'NA'

        UNION ALL


        SELECT
            away_team AS equipe,
            CAST(away_score AS INT) AS buts_pour,
            CAST(home_score AS INT) AS buts_contre
        FROM ma_table
        WHERE home_score != 'NA' AND away_score != 'NA'
    )
    GROUP BY equipe
    ORDER BY goal_average DESC
""")
nbrMatch.show()

+-----------+-------------+---------------+------------+
|     equipe|total_marques|total_encaisses|goal_average|
+-----------+-------------+---------------+------------+
|     Brazil|         2228|            910|        1318|
|    England|         2282|           1014|        1268|
|    Germany|         2205|           1133|        1072|
|  Argentina|         1880|           1042|         838|
|      Spain|         1464|            650|         814|
|South Korea|         1576|            808|         768|
|     Sweden|         2060|           1347|         713|
|Netherlands|         1694|           1000|         694|
|      Italy|         1441|            803|         638|
|     Mexico|         1592|            976|         616|
|       Iran|          968|            412|         556|
|     Russia|         1229|            696|         533|
|    Hungary|         1948|           1429|         519|
|  Australia|         1081|            583|         498|
|     France|         1560|    

In [ ]:
#17.Classement des équipes par nombre de victoires par année (ranking avec ROW_NUMBER).
nbrMatch = spark.sql("""
  WITH victorieux AS (

    SELECT
        YEAR(date) AS annee,
        CASE
            WHEN home_score > away_score THEN home_team
            WHEN away_score > home_score THEN away_team
        END AS equipe
    FROM ma_table
    WHERE home_score != away_score -- On exclut les matchs nuls
),
comptage_victoires AS (
    -- On compte les victoires par année et par équipe
    SELECT
        annee,
        equipe,
        COUNT(*) AS total_victoires
    FROM victorieux
    GROUP BY annee, equipe
)

SELECT
    annee,
    equipe,
    total_victoires,
    ROW_NUMBER() OVER(PARTITION BY annee ORDER BY total_victoires DESC) as rang
FROM comptage_victoires
ORDER BY annee DESC, rang ASC
""")
nbrMatch.show()

+-----+------------+---------------+----+
|annee|      equipe|total_victoires|rang|
+-----+------------+---------------+----+
| 2022| South Korea|              9|   1|
| 2022|  Costa Rica|              9|   2|
| 2022|     Bahrain|              9|   3|
| 2022|  Uzbekistan|              8|   4|
| 2022|   Argentina|              8|   5|
| 2022|     Senegal|              8|   6|
| 2022|     Algeria|              8|   7|
| 2022|     Tunisia|              8|   8|
| 2022|      Mexico|              8|   9|
| 2022|     Morocco|              7|  10|
| 2022|Burkina Faso|              7|  11|
| 2022|      Brazil|              7|  12|
| 2022|     Uruguay|              7|  13|
| 2022|       Egypt|              7|  14|
| 2022|    Thailand|              7|  15|
| 2022|      Jordan|              7|  16|
| 2022|    Cameroon|              6|  17|
| 2022|      Canada|              6|  18|
| 2022|   Indonesia|              6|  19|
| 2022|      Greece|              6|  20|
+-----+------------+--------------

In [ ]:
#18.Évolution du nombre de matchs par décennie.
nbrMatch = spark.sql("""
 SELECT
    FLOOR(YEAR(date) / 10) * 10 AS decennie,
    COUNT(*) AS nombre_de_matchs
FROM ma_table
GROUP BY decennie
ORDER BY decennie ASC;
""")
nbrMatch.show()

+--------+----------------+
|decennie|nombre_de_matchs|
+--------+----------------+
|    1870|              13|
|    1880|              55|
|    1890|              59|
|    1900|             124|
|    1910|             283|
|    1920|             729|
|    1930|            1010|
|    1940|             804|
|    1950|            1576|
|    1960|            2810|
|    1970|            3922|
|    1980|            4795|
|    1990|            6596|
|    2000|            9422|
|    2010|            9735|
|    2020|            2219|
+--------+----------------+



In [ ]:
#19.Identifier les équipes invaincues sur une année donnée.
nbrMatch = spark.sql("""
 WITH equipes_de_l_annee AS (

    SELECT home_team AS equipe FROM ma_table WHERE YEAR(date) = 2004
    UNION
    SELECT away_team AS equipe FROM ma_table WHERE YEAR(date) = 2004
),
perdants AS (

    SELECT away_team AS equipe FROM ma_table WHERE YEAR(date) = 2004 AND away_score < home_score
    UNION
    SELECT home_team AS equipe FROM ma_table WHERE YEAR(date) = 2004 AND home_score < away_score
)

SELECT equipe
FROM equipes_de_l_annee
EXCEPT
SELECT equipe
FROM perdants;
""")
nbrMatch.show()

+--------------+
|        equipe|
+--------------+
|       Lesotho|
|          Cuba|
|Basque Country|
|     Andalusia|
|      Ethiopia|
|        Kernow|
| French Guiana|
+--------------+



In [ ]:
#20.Calculer la longue série de victoires consécutives par équipe.
nbrMatch = spark.sql("""
WITH resultats AS (
    SELECT date, home_team AS equipe, CASE WHEN home_score > away_score THEN 'V' ELSE 'X' END AS res FROM ma_table
    UNION ALL
    SELECT date, away_team AS equipe, CASE WHEN away_score > home_score THEN 'V' ELSE 'X' END AS res FROM ma_table
),
groupes AS (
    SELECT equipe, res,
           ROW_NUMBER() OVER(PARTITION BY equipe ORDER BY date)
           - ROW_NUMBER() OVER(PARTITION BY equipe, res ORDER BY date) AS grp
    FROM resultats
)
SELECT distinct equipe, COUNT(*) AS longue_serie
FROM groupes
WHERE res = 'V'
GROUP BY equipe, grp
ORDER BY longue_serie DESC;
""")
nbrMatch.show()

+-------------+------------+
|       equipe|longue_serie|
+-------------+------------+
|      Padania|          15|
|        Spain|          15|
|       Brazil|          14|
|       France|          14|
|    Australia|          13|
|       Brazil|          13|
|       Guyana|          13|
|     Scotland|          13|
|       Mexico|          13|
|        Italy|          13|
|      Morocco|          12|
|    Indonesia|          12|
|       Russia|          12|
|       France|          12|
|      Belgium|          12|
|    German DR|          12|
|      Germany|          12|
|        Spain|          12|
|United States|          11|
|        Italy|          11|
+-------------+------------+
only showing top 20 rows


In [7]:
#21.Pour chaque tournoi, déterminer l’équipe la plus victorieuse.
nbrMatch = spark.sql("""WITH MatchWinners AS (

    SELECT
        tournament,
        CASE
            WHEN home_score > away_score THEN home_team
            WHEN away_score > home_score THEN away_team
            ELSE NULL -- On ignore les matchs nuls
        END AS winner
    FROM ma_table
),
VictoryCounts AS (

    SELECT
        tournament,
        winner as team,
        COUNT(*) as total_wins
    FROM MatchWinners
    WHERE winner IS NOT NULL
    GROUP BY tournament, winner
),
RankedTeams AS (

    SELECT
        tournament,
        team,
        total_wins,
        RANK() OVER (PARTITION BY tournament ORDER BY total_wins DESC) as rank
    FROM VictoryCounts
)

SELECT
    tournament,
    team,
    total_wins
FROM RankedTeams
WHERE rank = 1;
""")
nbrMatch.show()

+--------------------+-----------+----------+
|          tournament|       team|total_wins|
+--------------------+-----------+----------+
|     ABCS Tournament|   Suriname|         7|
|       AFC Asian Cup|       Iran|        41|
|AFC Asian Cup qua...|       Iran|        36|
|   AFC Challenge Cup| Tajikistan|        11|
|   AFC Challenge Cup|North Korea|        11|
|AFC Challenge Cup...|Philippines|         7|
|    AFF Championship|   Thailand|        42|
|AFF Championship ...|Timor-Leste|         2|
|AFF Championship ...|     Brunei|         2|
|African Cup of Na...|      Egypt|        60|
|African Cup of Na...|Ivory Coast|        67|
|African Nations C...|    Morocco|        11|
|African Nations C...|    Nigeria|        11|
|African Nations C...|     Uganda|         4|
|    Afro-Asian Games|      India|         2|
|  Amílcar Cabral Cup|    Senegal|        38|
|            Arab Cup|       Iraq|        13|
|Arab Cup qualific...|    Lebanon|         3|
|Arab Cup qualific...|      Syria|

In [8]:
#22.Comparer les performances à domicile vs à l’extérieur pour chaque équipe
nbrMatch = spark.sql("""WITH HomeStats AS (
    SELECT
        home_team AS team,
        COUNT(*) AS home_matches,
        SUM(CASE WHEN home_score > away_score THEN 1 ELSE 0 END) AS home_wins,
        SUM(CASE WHEN home_score = away_score THEN 1 ELSE 0 END) AS home_draws,
        SUM(CASE WHEN home_score < away_score THEN 1 ELSE 0 END) AS home_losses
    FROM ma_table
    GROUP BY home_team
),
AwayStats AS (
    SELECT
        away_team AS team,
        COUNT(*) AS away_matches,
        SUM(CASE WHEN away_score > home_score THEN 1 ELSE 0 END) AS away_wins,
        SUM(CASE WHEN away_score = home_score THEN 1 ELSE 0 END) AS away_draws,
        SUM(CASE WHEN away_score < home_score THEN 1 ELSE 0 END) AS away_losses
    FROM ma_table
    GROUP BY away_team
)
SELECT
    h.team,
    h.home_matches,
    h.home_wins,
    ROUND((h.home_wins / h.home_matches) * 100, 2) AS home_win_rate_pct,
    a.away_matches,
    a.away_wins,
    ROUND((a.away_wins / a.away_matches) * 100, 2) AS away_win_rate_pct
FROM HomeStats h
JOIN AwayStats a ON h.team = a.team
ORDER BY h.team;
""")
nbrMatch.show()

+-------------------+------------+---------+-----------------+------------+---------+-----------------+
|               team|home_matches|home_wins|home_win_rate_pct|away_matches|away_wins|away_win_rate_pct|
+-------------------+------------+---------+-----------------+------------+---------+-----------------+
|           Abkhazia|          18|        9|             50.0|          10|        3|             30.0|
|        Afghanistan|          42|       17|            40.48|          79|       15|            18.99|
|            Albania|         190|       68|            35.79|         171|       26|             15.2|
|           Alderney|           7|        0|              0.0|          12|        3|             25.0|
|            Algeria|         309|      173|            55.99|         238|       65|            27.31|
|     American Samoa|          24|        4|            16.67|          24|        1|             4.17|
|          Andalusia|          12|        7|            58.33|  